## notMNIST - CNN


In [1]:
from __future__ import print_function

import numpy as np
from scipy import ndimage
import pickle

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import advanced_activations
from keras import backend as K

import tensorflow as tf

# Checking tensorflow processing devices
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

Using TensorFlow backend.


[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14807064050257912011
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 1526107340
locality {
  bus_id: 1
}
incarnation: 16138876657672656534
physical_device_desc: "device: 0, name: GeForce GTX 680, pci bus id: 0000:01:00.0"
]


### Data Loading

In [5]:
pickle_file = 'C:/Users/Macal/Documents/Projects/Competitions/MNIST/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f,encoding='iso-8859-1')
    X_train = save['train_dataset']
    y_train = save['train_labels']
    X_validation = save['valid_dataset']
    y_validation = save['valid_labels']
    X_test = save['test_dataset']
    y_test = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', X_train.shape, y_train.shape)
    print('Validation set', X_validation.shape, y_validation.shape)
    print('Test set', X_test.shape, y_test.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [6]:
# input image dimensions
img_rows, img_cols = 28, 28

if K.image_data_format() == 'channels_first':
    X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    X_validation = X_validation.reshape(X_validation.shape[0], 1, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_validation = X_validation.reshape(X_validation.shape[0], img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

X_train = X_train.astype('float32')
X_validation = X_validation.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_validation /= 255
X_test /= 255

print('X_train shape:', X_train.shape)
print('y_train shape:', y_train.shape)
print('X_validation shape:', X_validation.shape)
print('y_validation shape:', y_validation.shape)
print('X_test shape:', X_test.shape)
print('y_test shape:', y_test.shape)
print(X_train.shape[0], 'Train samples')
print(X_validation.shape[0], 'Validation samples')
print(X_test.shape[0], 'Test samples')

X_train shape: (200000, 28, 28, 1)
y_train shape: (200000,)
X_validation shape: (10000, 28, 28, 1)
y_validation shape: (10000,)
X_test shape: (10000, 28, 28, 1)
y_test shape: (10000,)
200000 Train samples
10000 Validation samples
10000 Test samples


### Data Augmentation

In [ ]:
# Augment training data
def augment_training_data(images, labels):
    """
    Generates

    Creates an additional 300,000 
    
    Takes ~1.25 minutes with an i7/16gb machine
    """

    # Empty lists to fill
    expanded_images = []
    expanded_labels = []

    # Looping through
    j = 0   # counter
    for x, y in zip(images, labels):
        j = j + 1
        if j % 10000 == 0:
            print('Expanding data: %03d / %03d' % (j, np.size(images, 0)))

        # register original data
        expanded_images.append(x)
        expanded_labels.append(y)

        # get a value for the background
        # zero is the expected value, but median() is used to estimate background's value
        bg_value = np.median(x)  # this is regarded as background's value
        image = np.reshape(x, (-1, 28))

        for i in range(4):
            # rotate the image with random degree
            angle = np.random.randint(-15, 15, 1)
            new_img = ndimage.rotate(
                image, angle, reshape=False, cval=bg_value)

            # shift the image with random distance
            shift = np.random.randint(-2, 2, 2)
            new_img_ = ndimage.shift(new_img, shift, cval=bg_value)

            # register new training data
            expanded_images.append(np.reshape(new_img_, (28, 28, 1)))
            expanded_labels.append(y)

    # images and labels are concatenated for random-shuffle at each epoch
    # notice that pair of image and label should not be broken
#     expanded_train_total_data = np.concatenate((expanded_images, expanded_labels), axis=1)
#     print(np.array(expanded_images).shape)
#     print(np.array(expanded_labels).shape)
#     np.random.shuffle(expanded_train_total_data)

#     return expanded_train_total_data
    return expanded_images, expanded_labels


print('Starting')
augmented = augment_training_data(X_train, y_train)
print('Completed')

# Appending to the end of the current X/y train
X_train_aug = np.append(X_train, augmented[0], axis=0)
y_train_aug = np.append(y_train, augmented[1])

# Saving as a numpy array
print('\nSaving NP arrays')
np.save('X_train_augmented.npy', X_train)
np.save('y_train_augmented.npy', y_train)
print('Completed')

print('\nX_train shape:', X_train_aug.shape)
print('y_train shape:', y_train_aug.shape)
print(X_train_aug.shape[0], 'Train samples')
print(X_validation.shape[0], 'Validation samples')
print(X_test.shape[0], 'Test samples')

#### Loading In Augmented Data

If not running the data augmentation code above

In [7]:
# Uncomment if not performing data augmentation here
X_train_aug = np.load('X_train_augmented.npy')
y_train_aug = np.load('y_train_augmented.npy')

print('X_train_aug shape:', X_train_aug.shape)
print('y_train_aug shape:', y_train_aug.shape)
print(X_train_aug.shape[0], 'Train samples')
print(X_validation.shape[0], 'Validation samples')
print(X_test.shape[0], 'Test samples')

X_train_aug shape: (1200000, 28, 28, 1)
y_train_aug shape: (1200000,)
1200000 Train samples
10000 Validation samples
10000 Test samples


### Model Training on Augmented Data

In [8]:
# One-hot encoding for keras input
num_classes = 10
y_train = keras.utils.np_utils.to_categorical(y_train, num_classes)
y_train_aug = keras.utils.np_utils.to_categorical(y_train_aug, num_classes)
y_validation = keras.utils.np_utils.to_categorical(y_validation, num_classes)
y_test = keras.utils.np_utils.to_categorical(y_test, num_classes)

In [18]:
batch_size = 128
epochs = 20

## Constructing the model
model = Sequential()

# First hidden layer
model.add(Conv2D(32, kernel_size=(5, 5),
                 input_shape=input_shape))
model.add(advanced_activations.LeakyReLU(alpha=0.3))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Second hidden layer
model.add(Conv2D(64, (5, 5)))
model.add(advanced_activations.LeakyReLU(alpha=0.3))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Third hidden layer
model.add(Flatten())
model.add(Dense(1024))
model.add(advanced_activations.LeakyReLU(alpha=0.3))
model.add(Dropout(0.5))

# Output Layer
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(X_train_aug, y_train_aug,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_validation, y_validation))

score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 1200000 samples, validate on 10000 samples
Epoch 1/20
1200000/1200000 [==============================] - 164s - loss: 0.6342 - acc: 0.8089 - val_loss: 0.4062 - val_acc: 0.8776
Epoch 2/20
1200000/1200000 [==============================] - 161s - loss: 0.4529 - acc: 0.8667 - val_loss: 0.3555 - val_acc: 0.8960
Epoch 3/20
1200000/1200000 [==============================] - 162s - loss: 0.4103 - acc: 0.8788 - val_loss: 0.3301 - val_acc: 0.8995
Epoch 4/20
1200000/1200000 [==============================] - 166s - loss: 0.3838 - acc: 0.8862 - val_loss: 0.3137 - val_acc: 0.9052
Epoch 5/20
1200000/1200000 [==============================] - 165s - loss: 0.3647 - acc: 0.8917 - val_loss: 0.3057 - val_acc: 0.9097
Epoch 6/20
1200000/1200000 [==============================] - 176s - loss: 0.3496 - acc: 0.8962 - val_loss: 0.2941 - val_acc: 0.9120
Epoch 7/20
1200000/1200000 [==============================] - 204s - loss: 0.3377 - acc: 0.8992 - val_loss: 0.2860 - val_acc: 0.9155
Epoch 8/20
120000

### Model Training on non-Augmented Data

In [10]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_validation, y_validation))

score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 200000 samples, validate on 10000 samples
Epoch 1/20
200000/200000 [==============================] - 30s - loss: 1.0068 - acc: 0.6808 - val_loss: 0.5768 - val_acc: 0.8252
Epoch 2/20
200000/200000 [==============================] - 27s - loss: 0.5396 - acc: 0.8430 - val_loss: 0.4852 - val_acc: 0.8557
Epoch 3/20
200000/200000 [==============================] - 28s - loss: 0.4784 - acc: 0.8609 - val_loss: 0.4461 - val_acc: 0.8699
Epoch 4/20
200000/200000 [==============================] - 27s - loss: 0.4457 - acc: 0.8709 - val_loss: 0.4164 - val_acc: 0.8745
Epoch 5/20
200000/200000 [==============================] - 28s - loss: 0.4236 - acc: 0.8770 - val_loss: 0.3951 - val_acc: 0.8835
Epoch 6/20
200000/200000 [==============================] - 27s - loss: 0.4059 - acc: 0.8818 - val_loss: 0.3808 - val_acc: 0.8862
Epoch 7/20
200000/200000 [==============================] - 27s - loss: 0.3922 - acc: 0.8859 - val_loss: 0.3729 - val_acc: 0.8877
Epoch 8/20
200000/200000 [=============